In [1]:
from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
from sklearn.model_selection import train_test_split

import pandas as pd
from datasets import load_dataset, Dataset

In [2]:

df = pd.read_csv('srpski.csv', sep='\t')
df = df.drop(columns=['Rbr', 'SR', 'sr/sr', 'Naslov', 'Jezik'])

df['label'] = df['Autor'].astype('category').cat.codes
df = df.drop(columns=['Autor'])
df = df.rename(columns={'Tekst':'text'})

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 56
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 15
    })
})

In [3]:
model_id = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model = SetFitModel.from_pretrained(model_id)
def make_model(params=None):
    return SetFitModel.from_pretrained(
        model_id
    )

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [4]:
trainer = SetFitTrainer(
    model_init=make_model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20,
    num_epochs=1,
    column_mapping={'text':"text", "label": "label"}
)

C:\Users\raden\AppData\Local\Temp\ipykernel_26084\1886594068.py:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/56 [00:00<?, ? examples/s]

In [5]:
def hyperparameter_search_function(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [4, 8, 16, 32])    }


In [6]:
trainer.train()

***** Running training *****
  Num examples = 140
  Num epochs = 1
  Total optimization steps = 140
  Total train batch size = 16


Step,Training Loss


In [7]:
metrics = trainer.evaluate()
metrics

***** Running evaluation *****


{'accuracy': 0.26666666666666666}